In [215]:
import json
from itertools import chain
from pprint import pprint
from time import time

import numpy as np

from gensim.models import Word2Vec
from gensim.corpora.dictionary import Dictionary

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout
np.random.seed(1337)

In [89]:
def index_to_one_hot_encoding(index, vector_length):
    return [1 if i == index else 0 for i in xrange(vector_length)]

In [166]:
treebank_file1 = open('json/OPTA-treebank-0.1.json')
treebank_file2 = open('skladnica_output.json')
treebank = chain(list(json.load(treebank_file1)), list(json.load(treebank_file2)))

# treebank_file = open('json/OPTA-treebank-0.1.json')
# treebank_file = open('skladnica_output.json')
# treebank = json.load(treebank_file)

X = []
y = []
for entry in treebank:
    tree = entry['parsedSent']
    words = []
    sentiment = None
    for index, node in enumerate(tree):
        word = node.split('\t')[1].lower()
        words.append(word)
        if node.split('\t')[10] == 'S':
            sentiment = index
    if sentiment:
        X.append(words)
        y.append(index_to_one_hot_encoding(sentiment, len(words)))

# for a, b in zip(X, y):
#     print ' '.join(a)
#     print b

dataset_length = len(X)
slicing_point = int(dataset_length*0.9)

X_train = X[:slicing_point]
y_train = y[:slicing_point]
X_test = X[slicing_point+1:]
y_test = y[slicing_point+1:]

treebank_vocabulary = set(chain(*X))
print len(treebank_vocabulary)

3906


In [128]:

w2v_model = Word2Vec.load('w2v_allwiki_nkjp300_200.model')

In [131]:
w2v_vocabulary = Dictionary()
w2v_vocabulary.doc2bow(w2v_model.vocab.keys(), allow_update=True)


706935

In [143]:
w2indx = {v: k+1 for k, v in w2v_vocabulary.items()}
w2vec = {word: model[word] for word in w2indx.keys()}

In [165]:
w2v_vocabulary_size = len(w2indx) + 1
w2v_vocabulary_dimension = len(w2vec.values()[0])



In [172]:
our_dict = {}
missing_count = 0

for word in treebank_vocabulary:
    if word in w2v_model:
        our_dict[word] = w2v_model[word]
    else:
        missing_count += 1
#         print("missing key %s" %(word)) 
        our_dict[word] = np.zeros(w2v_vocabulary_dimension)
print missing_count

241


In [187]:
# Map treebank words to w2v indices

X_train_vec = []
for sentence in X_train:
    vectorized_sentence = []
    for word in sentence:
        try:
            vectorized_sentence.append(w2indx[word])
        except KeyError:
            vectorized_sentence.append(0)
    X_train_vec.append(vectorized_sentence)
    
pprint(X_train_vec[:5])


[[630112, 499270, 421911, 157116, 0],
 [258612, 180367, 193758, 553222, 276450, 0],
 [580109, 431241, 193758, 639684, 453311, 0],
 [431241, 633875, 585097, 123882, 0, 513739, 0],
 [446500, 212238, 536117, 211923, 428294, 426621, 0]]


In [195]:
embedding_weights = np.zeros((w2v_vocabulary_size , w2v_vocabulary_dimension))
for word, index in w2indx.items():
    embedding_weights[index, :] = w2vec[word]

In [211]:
input_length = 100
X_train = sequence.pad_sequences(X_train_vec, maxlen=input_length)
# X_test = sequence.pad_sequences(X_test, maxlen=input_length)

model = Sequential()
model.add(Embedding(output_dim=w2v_vocabulary_dimension,
                    input_dim=w2v_vocabulary_size,
                    mask_zero=True,
                    weights=[embedding_weights],
                    input_length=input_length))


In [213]:
model.add(LSTM(w2v_vocabulary_dimension))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
          
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              class_mode='binary')

/home/tomek/.virtualenvs/deeplearning/local/lib/python2.7/site-packages/keras/models.py:325: UserWarning: "class_mode" argument is deprecated, please remove it.
  warnings.warn('"class_mode" argument is deprecated, '


In [217]:
batch_size = 32
n_epoch = 5

print("Train...")
start_time = time()
hist = model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=n_epoch,
          verbose=2, show_accuracy=True)
end_time = time()

print("Training took %f secs" %(end_time - start_time))

Train...


/home/tomek/.virtualenvs/deeplearning/local/lib/python2.7/site-packages/keras/models.py:385: UserWarning: The "show_accuracy" argument is deprecated, instead you should pass the "accuracy" metric to the model at compile time:
`model.compile(optimizer, loss, metrics=["accuracy"])`
  warnings.warn('The "show_accuracy" argument is deprecated, '


Epoch 1/5


ValueError: ('Bad input argument to theano function with name "/home/tomek/.virtualenvs/deeplearning/local/lib/python2.7/site-packages/keras/backend/theano_backend.py:489"  at index 1(0-based)', 'setting an array element with a sequence.')